In [21]:
import pandas as pd
import os
import numpy as np

In [2]:
def load_csvs(directory):
    # Create an empty dictionary to store the dataframes
    dataframes = {}

    # Loop through each file in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):  # Check if the file is a CSV
            file_path = os.path.join(directory, filename)
            df_name = filename[:-4]  # Remove the '.csv' from the filename to use as the DataFrame name
            dataframes[df_name] = pd.read_csv(file_path)  # Read the CSV file and store it in the dictionary

    return dataframes

# Specify the directory containing your CSV files
directory = '../data/glycemic_events/'

# Load all CSV files into a dictionary
results_dict = load_csvs(directory)


### Expand dfs

In [3]:
original_df = results_dict['hypo_glc']
cut_df = results_dict['hypo_cut_glc']



In [40]:
original_df

,ID,Unnamed: 1,start_time,duration,end_time,lv2,prolonged,unique_id_orig
0,helm_1,0,2020-05-12 04:01:19,0 days 00:20:00,2020-05-12 04:21:19,False,False,1
1,helm_1,1,2020-05-12 04:41:19,0 days 01:25:01,2020-05-12 06:06:20,False,False,2
2,helm_1,2,2020-05-12 20:21:21,0 days 00:20:00,2020-05-12 20:41:21,False,False,3
3,helm_1,3,2020-05-15 14:21:28,0 days 00:20:00,2020-05-15 14:41:28,False,False,4
4,helm_1,4,2020-05-18 05:56:34,0 days 00:35:00,2020-05-18 06:31:34,False,False,5
...,...,...,...,...,...,...,...,...
11106,helm_988,28,2021-04-14 15:11:32,0 days 00:40:01,2021-04-14 15:51:33,False,False,11107
11107,helm_988,29,2021-04-15 08:06:35,0 days 00:40:01,2021-04-15 08:46:36,False,False,11108
11108,helm_988,30,2021-04-15 10:26:36,0 days 00:39:59,2021-04-15 11:06:35,False,False,11109
11109,helm_988,31,2021-04-15 18:56:36,0 days 00:30:00,2021-04-15 19:26:36,False,False,11110


In [7]:
cut_df['start_time'] = pd.to_datetime(cut_df['start_time'])
cut_df['end_time'] = pd.to_datetime(cut_df['end_time'])

### Add true negatives to dfs

In [4]:
import pandas as pd

def expand_data(df):
    # Convert 'start_time' and 'end_time' to datetime if they are not already
    df['start_time'] = pd.to_datetime(df['start_time'])
    df['end_time'] = pd.to_datetime(df['end_time'])
    
    # Create a new column 'lv1' and set it to True for all original rows
    df['lv1'] = True

    # Create an empty list to hold the new rows
    new_rows = []

    # Sort dataframe by start_time to ensure correct sequential processing
    df = df.sort_values(by='start_time').reset_index(drop=True)

    # Iterate over the dataframe, skip the first row for new row insertion
    for index in range(1, len(df)):
        previous_row = df.iloc[index - 1]
        current_row = df.iloc[index]

        # Define the new row's start and end times
        new_start_time = previous_row['end_time'] + pd.Timedelta(minutes=1)
        new_end_time = current_row['start_time'] - pd.Timedelta(minutes=1)

        # Ensure that the new end time is greater than the new start time before adding the row
        if new_start_time < new_end_time:
            new_row = {
                'ID': previous_row['ID'],  # Assume ID continuity, or modify as necessary
                'start_time': new_start_time,
                'end_time': new_end_time,
                'duration': new_end_time - new_start_time,
                'lv1': False,
                'lv2': False,
                # Add other necessary fields if required
            }
            new_rows.append(new_row)

    # Convert the list of new rows into a DataFrame
    new_rows_df = pd.DataFrame(new_rows)
    
    # Concatenate the original DataFrame with the new rows DataFrame
    expanded_df = pd.concat([df, new_rows_df], ignore_index=True).sort_values(by='start_time')
    expanded_df = expanded_df[['ID', 'start_time', 'end_time', 'duration', 'lv1', 'lv2']]
    expanded_df = expanded_df.sort_values(by='start_time')
    return expanded_df


In [5]:
original_df_expanded = original_df.groupby('ID').apply(expand_data).reset_index(drop=True)

/tmp/ipykernel_20644/3913728291.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['start_time'] = pd.to_datetime(df['start_time'])
/tmp/ipykernel_20644/531986263.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  original_df_expanded = original_df.groupby('ID').apply(expand_data).reset_index(drop=True)


In [6]:
#cut_df_expanded = cut_df.groupby('ID').apply(expand_data).reset_index(drop=True)

In [7]:
original_df_expanded

,ID,start_time,end_time,duration,lv1,lv2
0,helm_1,2020-05-12 04:01:19,2020-05-12 04:21:19,0 days 00:20:00,True,False
1,helm_1,2020-05-12 04:22:19,2020-05-12 04:40:19,0 days 00:18:00,False,False
2,helm_1,2020-05-12 04:41:19,2020-05-12 06:06:20,0 days 01:25:01,True,False
3,helm_1,2020-05-12 06:07:20,2020-05-12 20:20:21,0 days 14:13:01,False,False
4,helm_1,2020-05-12 20:21:21,2020-05-12 20:41:21,0 days 00:20:00,True,False
...,...,...,...,...,...,...
21730,helm_988,2021-04-15 10:26:36,2021-04-15 11:06:35,0 days 00:39:59,True,False
21731,helm_988,2021-04-15 11:07:35,2021-04-15 18:55:36,0 days 07:48:01,False,False
21732,helm_988,2021-04-15 18:56:36,2021-04-15 19:26:36,0 days 00:30:00,True,False
21733,helm_988,2021-04-15 19:27:36,2021-04-16 17:40:39,0 days 22:13:03,False,False


In [16]:
def calculate_confusion_matrix(original_df, cut_df):
    TP_lv1 = 0
    TP_lv2 = 0
    FP_lv1 = 0
    FP_lv2 = 0
    FN_lv1 = 0
    FN_lv2 = 0
    TN = 0

    used_cut_ids = set()

    for i, orig_row in original_df.iterrows():
        overlaps = cut_df[(cut_df['start_time'] < orig_row['end_time']) & (cut_df['end_time'] > orig_row['start_time'])]
        valid_overlaps = overlaps[~overlaps.index.isin(used_cut_ids)]

        if orig_row['lv1']:
            if valid_overlaps.empty:
                FN_lv1 += 1
                if orig_row['lv2']:
                    FN_lv2 += 1
            else:
                TP_lv1 += 1
                used_cut_ids.update(valid_overlaps.index)
                if orig_row['lv2'] and valid_overlaps['lv2'].any():
                    TP_lv2 += 1
        else:
            if valid_overlaps.empty:
                TN += 1
            else:
                FP_lv1 += len(valid_overlaps)
                if valid_overlaps['lv2'].any():
                    FP_lv2 += valid_overlaps['lv2'].sum()

    remaining_cuts = cut_df.loc[~cut_df.index.isin(used_cut_ids)]
    FP_lv1 += len(remaining_cuts)
    FP_lv2 += remaining_cuts['lv2'].sum()

    return {
        'TP_lv1': TP_lv1,
        'TP_lv2': TP_lv2,
        'FP_lv1': FP_lv1,
        'FP_lv2': FP_lv2,
        'FN_lv1': FN_lv1,
        'FN_lv2': FN_lv2,
        'TN': TN
    }

# Assuming original_df_expanded and cut_df are already defined and preprocessed correctly
confusion_matrix = calculate_confusion_matrix(original_df_expanded, cut_df)
print(confusion_matrix)


{'TP_lv1': 5891, 'TP_lv2': 813, 'FP_lv1': 44958, 'FP_lv2': 6580, 'FN_lv1': 5220, 'FN_lv2': 859, 'TN': 4485}


In [11]:
10232+879

11111

In [12]:
1612+467

2079

In [14]:
original_df.lv2.sum()

2079

### Merge dfs

In [18]:

# Sample DataFrames
# Ensure your dataframes are named original_df and comparison_df

# Adding unique identifiers
original_df_expanded['unique_id_orig'] = range(1, len(original_df_expanded) + 1)
cut_df['unique_id_cut'] = range(1, len(cut_df) + 1)

# Convert dates to datetime if they aren't already
original_df_expanded['start_time'] = pd.to_datetime(original_df_expanded['start_time'])
original_df_expanded['end_time'] = pd.to_datetime(original_df_expanded['end_time'])
cut_df['start_time'] = pd.to_datetime(cut_df['start_time'])
cut_df['end_time'] = pd.to_datetime(cut_df['end_time'])


In [38]:
import pandas as pd

def full_outer_join_custom_per_id(original_df_expanded, cut_df):
    all_rows = []

    # Get unique IDs from both dataframes
    all_ids = set(original_df_expanded['ID']).union(set(cut_df['ID']))

    # Process each ID separately
    for id in all_ids:
        orig_group = original_df_expanded[original_df_expanded['ID'] == id]
        cut_group = cut_df[cut_df['ID'] == id]

        # Tracking matched cut IDs to handle unmatched cut rows later
        matched_cut_ids = set()

        # Check for overlaps within the same ID group
        for _, orig_row in orig_group.iterrows():
            matched = False
            for _, cut_row in cut_group.iterrows():
                if orig_row['end_time'] >= cut_row['start_time'] and orig_row['start_time'] <= cut_row['end_time']:
                    all_rows.append({
                        'id': id,
                        'original_id': orig_row['unique_id_orig'],
                        'cut_id': cut_row['unique_id_cut'],
                        'original_start_time': orig_row['start_time'],
                        'original_end_time': orig_row['end_time'],
                        'original_lv1': orig_row['lv1'],
                        'original_lv2': orig_row['lv2'],
                        'cut_start_time': cut_row['start_time'],
                        'cut_end_time': cut_row['end_time'],
                        'cut_lv1': True,
                        'cut_lv2': cut_row['lv2'],
                    })
                    matched = True
                    matched_cut_ids.add(cut_row['unique_id_cut'])
            if not matched:
                all_rows.append({
                    'id': id,
                    'original_id': orig_row['unique_id_orig'],
                    'cut_id': None,
                    'original_start_time': orig_row['start_time'],
                    'original_end_time': orig_row['end_time'],
                    'original_lv1': orig_row['lv1'],
                    'original_lv2': orig_row['lv2'],
                    'cut_start_time': None,
                    'cut_end_time': None,
                    'cut_lv1': False,
                    'cut_lv2': False,
                })

        # Add cut rows that were not matched with any original rows
        for _, cut_row in cut_group.iterrows():
            if cut_row['unique_id_cut'] not in matched_cut_ids:
                all_rows.append({
                    'id': id,
                    'original_id': None,
                    'cut_id': cut_row['unique_id_cut'],
                    'original_start_time': None,
                    'original_end_time': None,
                    'original_lv1': False,
                    'original_lv2': False,
                    'cut_start_time': cut_row['start_time'],
                    'cut_end_time': cut_row['end_time'],
                    'cut_lv1': True,
                    'cut_lv2': cut_row['lv2'],
                })

    # Convert the list to a DataFrame
    result_df = pd.DataFrame(all_rows)

    # Convert IDs to integers, handling NaN where necessary
    result_df['original_id'] = result_df['original_id'].astype('Int64')
    result_df['cut_id'] = result_df['cut_id'].astype('Int64')

    return result_df

# To apply the function, ensure you have dataframes `original_df_expanded` and `cut_df` defined with appropriate data
# Example of application:
full_outer_result = full_outer_join_custom_per_id(original_df_expanded, cut_df)
# print(full_outer_result)


In [26]:
full_outer_result.to_csv('../data/joined_results/cut_glc.csv', index=False)

In [29]:
full_outer_result[(full_outer_result['Original LV1'] == False) & (full_outer_result['Cut LV1'] == True)]

,ID,original_id,cut_id,Original Start Time,Original End Time,Original LV1,Original LV2,Cut Start Time,Cut End Time,Cut LV1,Cut LV2,Overlap
1,helm_756,19319,7979,2020-06-06 01:55:12,2020-06-07 09:38:16,False,False,2020-06-06 00:19:11,2020-06-06 02:04:12,True,False,True
3,helm_756,19321,7980,2020-06-07 10:45:16,2020-06-07 18:43:18,False,False,2020-06-07 09:49:16,2020-06-07 10:49:16,True,False,True
5,helm_756,19323,7981,2020-06-07 19:30:19,2020-06-09 12:28:23,False,False,2020-06-07 18:49:19,2020-06-07 19:34:19,True,True,True
7,helm_756,19325,7982,2020-06-09 13:15:24,2020-06-09 18:23:25,False,False,2020-06-09 12:34:23,2020-06-09 13:19:23,True,False,True
9,helm_756,19327,7983,2020-06-09 19:10:25,2020-06-09 20:38:25,False,False,2020-06-09 18:34:25,2020-06-09 19:19:25,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
21829,helm_573,16607,6874,2020-08-20 04:19:10,2020-08-20 17:07:10,False,False,2020-08-20 03:43:09,2020-08-20 04:28:09,True,True,True
21831,helm_573,16609,6875,2020-08-20 17:39:10,2020-08-23 02:22:13,False,False,2020-08-20 17:13:10,2020-08-20 17:43:10,True,False,True
21833,helm_573,16611,6876,2020-08-23 03:04:13,2020-08-24 05:07:18,False,False,2020-08-23 02:28:13,2020-08-23 03:13:17,True,False,True
21837,helm_573,16615,6878,2020-08-25 17:04:16,2020-08-29 23:27:20,False,False,2020-08-25 16:43:16,2020-08-25 17:13:16,True,False,True


In [84]:
full_outer_result

,id,original_id,cut_id,original_start_time,original_end_time,original_lv1,original_lv2,cut_start_time,cut_end_time,cut_lv1,cut_lv2
0,helm_756,19318,7979,2020-06-06 00:14:11,2020-06-06 01:54:12,True,False,2020-06-06 00:19:11,2020-06-06 02:04:12,True,False
1,helm_756,19319,7979,2020-06-06 01:55:12,2020-06-07 09:38:16,False,False,2020-06-06 00:19:11,2020-06-06 02:04:12,True,False
2,helm_756,19320,7980,2020-06-07 09:39:16,2020-06-07 10:44:16,True,False,2020-06-07 09:49:16,2020-06-07 10:49:16,True,False
3,helm_756,19321,7980,2020-06-07 10:45:16,2020-06-07 18:43:18,False,False,2020-06-07 09:49:16,2020-06-07 10:49:16,True,False
4,helm_756,19322,7981,2020-06-07 18:44:18,2020-06-07 19:29:19,True,True,2020-06-07 18:49:19,2020-06-07 19:34:19,True,True
...,...,...,...,...,...,...,...,...,...,...,...
21836,helm_573,16614,6878,2020-08-25 16:38:16,2020-08-25 17:03:16,True,False,2020-08-25 16:43:16,2020-08-25 17:13:16,True,False
21837,helm_573,16615,6878,2020-08-25 17:04:16,2020-08-29 23:27:20,False,False,2020-08-25 16:43:16,2020-08-25 17:13:16,True,False
21838,helm_573,16616,6879,2020-08-29 23:28:20,2020-08-30 00:03:20,True,False,2020-08-29 23:28:20,2020-08-30 00:13:21,True,False
21839,helm_573,16617,6879,2020-08-30 00:04:20,2020-08-31 02:06:16,False,False,2020-08-29 23:28:20,2020-08-30 00:13:21,True,False


In [85]:
import pandas as pd

def assign_overlap_ids(df):
    # First, sort the DataFrame to maintain a consistent order for processing
    df = df.sort_values(by=['original_id', 'cut_id', 'original_start_time'])

    # Create a unique key for each combination of 'original_id' and 'cut_id'
    # This considers rows where either 'original_id' or 'cut_id' might be leading to multiple overlaps
    df['unique_group'] = pd.factorize(df['original_id'].astype(str) + '-' + df['cut_id'].astype(str))[0] + 1

    # Sort by this new group key to ensure consistent group ordering
    df = df.sort_values(by='unique_group')

    # Assign unique overlap IDs based on the 'unique_group' change
    df['overlap_id'] = (df['unique_group'] != df['unique_group'].shift(1)).cumsum()

    # Drop the temporary 'unique_group' as it's no longer needed
    df = df.drop(columns=['unique_group'])

    return df

# Usage example
# Assuming 'merged_df' is your DataFrame containing 'original_id' and 'cut_id' among other necessary columns
# result_df = assign_overlap_ids(merged_df)
# print(result_df[['ID', 'original_id', 'cut_id', 'overlap_id']])


In [86]:
result_df = assign_overlap_ids(full_outer_result)


In [87]:
result_df

,id,original_id,cut_id,original_start_time,original_end_time,original_lv1,original_lv2,cut_start_time,cut_end_time,cut_lv1,cut_lv2,overlap_id
7881,helm_1,1,1,2020-05-12 04:01:19,2020-05-12 04:21:19,True,False,2020-05-12 04:01:19,2020-05-12 06:16:20,True,False,1
7882,helm_1,2,1,2020-05-12 04:22:19,2020-05-12 04:40:19,False,False,2020-05-12 04:01:19,2020-05-12 06:16:20,True,False,2
7883,helm_1,3,1,2020-05-12 04:41:19,2020-05-12 06:06:20,True,False,2020-05-12 04:01:19,2020-05-12 06:16:20,True,False,3
7884,helm_1,4,1,2020-05-12 06:07:20,2020-05-12 20:20:21,False,False,2020-05-12 04:01:19,2020-05-12 06:16:20,True,False,4
7885,helm_1,5,<NA>,2020-05-12 20:21:21,2020-05-12 20:41:21,True,False,NaT,NaT,False,False,5
...,...,...,...,...,...,...,...,...,...,...,...,...
6177,helm_1120,<NA>,678,NaT,NaT,False,False,2020-06-21 03:29:32,2020-06-21 03:59:32,True,False,21837
1438,helm_202,<NA>,4837,NaT,NaT,False,False,2020-08-28 19:48:56,2020-08-28 20:18:56,True,False,21838
13403,helm_671,<NA>,7536,NaT,NaT,False,False,2020-10-29 12:40:25,2020-10-29 13:10:25,True,False,21839
8529,helm_830,<NA>,8337,NaT,NaT,False,False,2019-10-19 22:21:22,2019-10-19 22:51:22,True,False,21840


In [81]:
import pandas as pd

def full_outer_join_custom_per_id(original_df_expanded, cut_df):
    all_rows = []
    overlap_counter = 1  # Counter to assign unique overlap IDs
    id_to_overlap_id = {}  # Map both original_id and cut_id to overlap_id

    # Get unique IDs from both dataframes
    all_ids = set(original_df_expanded['ID']).union(set(cut_df['ID']))

    # Process each ID separately
    for id in all_ids:
        orig_group = original_df_expanded[original_df_expanded['ID'] == id]
        cut_group = cut_df[cut_df['ID'] == id]

        # Tracking matched cut IDs to handle unmatched cut rows later
        matched_cut_ids = set()

        # Check for overlaps within the same ID group
        for _, orig_row in orig_group.iterrows():
            for _, cut_row in cut_group.iterrows():
                # Define keys for lookup
                orig_key = (id, orig_row['unique_id_orig'])
                cut_key = (id, cut_row['unique_id_cut'])
                
                # Check if there is an overlap
                if orig_row['end_time'] >= cut_row['start_time'] and orig_row['start_time'] <= cut_row['end_time']:
                    # Check if either the original_id or cut_id already has an assigned overlap_id
                    if orig_key in id_to_overlap_id:
                        overlap_id = id_to_overlap_id[orig_key]
                    elif cut_key in id_to_overlap_id:
                        overlap_id = id_to_overlap_id[cut_key]
                    else:
                        overlap_id = overlap_counter
                        overlap_counter += 1

                    # Map both original_id and cut_id to the same overlap_id
                    id_to_overlap_id[orig_key] = overlap_id
                    id_to_overlap_id[cut_key] = overlap_id
                    
                    # Add the row with the determined overlap_id
                    all_rows.append({
                        'ID': id,
                        'original_id': orig_row['unique_id_orig'],
                        'cut_id': cut_row['unique_id_cut'],
                        'original_start_time': orig_row['start_time'],
                        'original_end_time': orig_row['end_time'],
                        'original_lv1': orig_row['lv1'],
                        'original_lv2': orig_row['lv2'],
                        'cut_start_time': cut_row['start_time'],
                        'cut_end_time': cut_row['end_time'],
                        'cut_lv1': True,
                        'cut_lv2': cut_row['lv2'],
                        'overlap': True,
                        'overlap_id': overlap_id
                    })
                    matched_cut_ids.add(cut_row['unique_id_cut'])

        # Handle non-matching rows
        for _, orig_row in orig_group.iterrows():
            orig_key = (id, orig_row['unique_id_orig'])
            if orig_key not in id_to_overlap_id:
                overlap_id = overlap_counter
                overlap_counter += 1
                id_to_overlap_id[orig_key] = overlap_id
                all_rows.append({
                    'ID': id,
                    'original_id': orig_row['unique_id_orig'],
                    'cut_id': None,
                    'original_start_time': orig_row['start_time'],
                    'original_end_time': orig_row['end_time'],
                    'original_lv1': orig_row['lv1'],
                    'original_lv2': orig_row['lv2'],
                    'cut_start_time': None,
                    'cut_end_time': None,
                    'cut_lv1': False,
                    'cut_lv2': False,
                    'overlap': False,
                    'overlap_id': overlap_id
                })

        # Handle unmatched cut rows
        for _, cut_row in cut_group.iterrows():
            cut_key = (id, cut_row['unique_id_cut'])
            if cut_key not in id_to_overlap_id:
                overlap_id = overlap_counter
                overlap_counter += 1
                id_to_overlap_id[cut_key] = overlap_id
                all_rows.append({
                    'ID': id,
                    'original_id': None,
                    'cut_id': cut_row['unique_id_cut'],
                    'original_start_time': None,
                    'original_end_time': None,
                    'original_lv1': False,
                    'original_lv2': False,
                    'cut_start_time': cut_row['start_time'],
                    'cut_end_time': cut_row['end_time'],
                    'cut_lv1': True,
                    'cut_lv2': cut_row['lv2'],
                    'overlap': False,
                    'overlap_id': overlap_id
                })

    # Convert the list to a DataFrame
    result_df = pd.DataFrame(all_rows)

    # Convert IDs to integers, handling NaN where necessary
    result_df['original_id'] = result_df['original_id'].astype('Int64')
    result_df['cut_id'] = result_df['cut_id'].astype('Int64')

    return result_df

# Example of application (ensure you have original_df_expanded and cut_df with correct data):
# full_outer_result = full_outer_join_custom_per_id(original_df_expanded, cut_df)
# print(full_outer_result)


In [82]:
merged_df = full_outer_join_custom_per_id(original_df_expanded, cut_df)

In [83]:
merged_df

,ID,original_id,cut_id,original_start_time,original_end_time,original_lv1,original_lv2,cut_start_time,cut_end_time,cut_lv1,cut_lv2,overlap,overlap_id
0,helm_756,19318,7979,2020-06-06 00:14:11,2020-06-06 01:54:12,True,False,2020-06-06 00:19:11,2020-06-06 02:04:12,True,False,True,1
1,helm_756,19319,7979,2020-06-06 01:55:12,2020-06-07 09:38:16,False,False,2020-06-06 00:19:11,2020-06-06 02:04:12,True,False,True,1
2,helm_756,19320,7980,2020-06-07 09:39:16,2020-06-07 10:44:16,True,False,2020-06-07 09:49:16,2020-06-07 10:49:16,True,False,True,2
3,helm_756,19321,7980,2020-06-07 10:45:16,2020-06-07 18:43:18,False,False,2020-06-07 09:49:16,2020-06-07 10:49:16,True,False,True,2
4,helm_756,19322,7981,2020-06-07 18:44:18,2020-06-07 19:29:19,True,True,2020-06-07 18:49:19,2020-06-07 19:34:19,True,True,True,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21828,helm_573,16617,6879,2020-08-30 00:04:20,2020-08-31 02:06:16,False,False,2020-08-29 23:28:20,2020-08-30 00:13:21,True,False,True,15307
21829,helm_573,16618,6880,2020-08-31 02:07:16,2020-08-31 02:42:16,True,False,2020-08-31 02:17:16,2020-08-31 02:47:16,True,False,True,15308
21830,helm_573,16602,<NA>,2020-08-17 18:33:06,2020-08-17 18:53:07,True,False,NaT,NaT,False,False,False,15309
21831,helm_573,16603,<NA>,2020-08-17 18:54:07,2020-08-19 16:52:09,False,False,NaT,NaT,False,False,False,15310


In [76]:
merged_df.sort_values(['ID', 'original_start_time', 'cut_start_time'])

KeyError: 'ID'

In [71]:
merged_df.sort_values('overlap_id')

,ID,original_id,cut_id,original_start_time,original_end_time,original_lv1,original_lv2,cut_start_time,cut_end_time,cut_lv1,cut_lv2,overlap,overlap_id
0,helm_1,<NA>,<NA>,None,None,None,None,None,None,None,None,False,1
30,helm_1,<NA>,<NA>,None,None,None,None,None,None,None,None,False,1
31,helm_1,<NA>,<NA>,None,None,None,None,None,None,None,None,False,1
34,helm_1,<NA>,<NA>,None,None,None,None,None,None,None,None,False,1
35,helm_1,<NA>,<NA>,None,None,None,None,None,None,None,None,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22394,helm_1120,<NA>,<NA>,None,None,None,None,None,None,None,None,False,15307
26553,helm_202,<NA>,<NA>,None,None,None,None,None,None,None,None,False,15308
29252,helm_671,<NA>,<NA>,None,None,None,None,None,None,None,None,False,15309
30053,helm_830,<NA>,<NA>,None,None,None,None,None,None,None,None,False,15310


In [43]:
def assess_cut_groups(df):

    # Cut groups
    cut_drop = df.drop_duplicates(subset='cut_id')
    
    og_lv1 = df['original_lv1'].sum()
    og_lv2 = df['original_lv2'].sum()
    og_negatives = df['original_lv1'].count() - og_lv1
    cut_lv1 = df['cut_lv1'].iloc[0].sum()
    cut_lv2 = df['cut_lv2'].iloc[0].sum()
    return pd.Series({
        'Original LV1': og_lv1,
        'Original LV2': og_lv2,
        'Original Negatives': og_negatives,
        'Cut LV1': cut_lv1,
        'Cut LV2': cut_lv2,
    })

In [45]:
somethin = full_outer_result.groupby('overlap_id').apply(assess_cut_groups)

/tmp/ipykernel_20644/4157579624.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  somethin = full_outer_result.groupby('cut_id').apply(assess_cut_groups)


In [ ]:
merged_df = full_outer_join_custom_per_id(original_df_expanded, cut_df)

In [46]:
somethin

,Original LV1,Original LV2,Original Negatives,Cut LV1,Cut LV2
cut_id,,,,,
1,2,0,2,1,0
2,1,0,0,1,0
3,1,0,1,1,0
4,1,0,0,1,0
5,2,0,1,1,0
...,...,...,...,...,...
8971,1,0,1,1,0
8972,1,0,0,1,0
8973,1,0,1,1,0


In [48]:
somethin['cut-og_lv1'] = somethin['Cut LV1'] - somethin['Original LV1']
somethin['cut-og_lv2'] = somethin['Cut LV2'] - somethin['Original LV2']
somethin['tnlv1'] = somethin['Original Negatives'] - somethin['cut-og_lv1']
somethin['tnlv2'] = somethin['Original Negatives'] - somethin['cut-og_lv2']

In [49]:
somethin

,Original LV1,Original LV2,Original Negatives,Cut LV1,Cut LV2,cut-og_lv1,cut-og_lv2,tnlv1,tnlv2
cut_id,,,,,,,,,
1,2,0,2,1,0,-1,0,3,2
2,1,0,0,1,0,0,0,0,0
3,1,0,1,1,0,0,0,1,1
4,1,0,0,1,0,0,0,0,0
5,2,0,1,1,0,-1,0,2,1
...,...,...,...,...,...,...,...,...,...
8971,1,0,1,1,0,0,0,1,1
8972,1,0,0,1,0,0,0,0,0
8973,1,0,1,1,0,0,0,1,1


### Confusion matrix

In [24]:
duplicate_cut_ids

0         True
1         True
2         True
3         True
4         True
         ...  
21836     True
21837     True
21838     True
21839     True
21840    False
Length: 21841, dtype: bool

In [23]:
# Find duplicate cut_ids
duplicate_cut_ids = full_outer_result.duplicated('cut_id', keep=False)

# Set the specific columns to NaN where cut_id is duplicated
full_outer_result.loc[full_outer_result, ['cut_id', 'Cut Start Time', 'Cut End Time', 'Cut LV2']] = np.nan


KeyError: "None of [Index([                                                                                     ('I', 'D'),\n                                               ('o', 'r', 'i', 'g', 'i', 'n', 'a', 'l', '_', 'i', 'd'),\n                                                                        ('c', 'u', 't', '_', 'i', 'd'),\n       ('O', 'r', 'i', 'g', 'i', 'n', 'a', 'l', ' ', 'S', 't', 'a', 'r', 't', ' ', 'T', 'i', 'm', 'e'),\n                 ('O', 'r', 'i', 'g', 'i', 'n', 'a', 'l', ' ', 'E', 'n', 'd', ' ', 'T', 'i', 'm', 'e'),\n                                          ('O', 'r', 'i', 'g', 'i', 'n', 'a', 'l', ' ', 'L', 'V', '1'),\n                                          ('O', 'r', 'i', 'g', 'i', 'n', 'a', 'l', ' ', 'L', 'V', '2'),\n                                ('C', 'u', 't', ' ', 'S', 't', 'a', 'r', 't', ' ', 'T', 'i', 'm', 'e'),\n                                          ('C', 'u', 't', ' ', 'E', 'n', 'd', ' ', 'T', 'i', 'm', 'e'),\n                                   

In [26]:
overlaps_df[overlaps_df['Cut Start Time'].isna()]

,original_id,cut_id,ID,Original Start Time,Original End Time,Orginal LV1,Original LV2,Cut Start Time,Cut End Time,Cut LV2,Overlap


In [ ]:
overlaps_df

In [ ]:
# Apply the overlap function
merged_df['overlaps'] = merged_df.apply(lambda x: overlaps(x[['start_time_x', 'end_time_x']], x[['start_time_y', 'end_time_y']]), axis=1)

# Filter to only overlapping intervals
overlapping_df = merged_df[merged_df['overlaps']]


In [ ]:
overlapping_df

### Calculate overlaps

In [88]:
import pandas as pd

def overlaps(row, df):
    """Helper function to find overlaps."""
    return df[(df['start_time'] < row['end_time']) & (df['end_time'] > row['start_time'])]

def analyze_episodes(original_df, comparison_df):
    # Ensure datetime conversion
    original_df['start_time'] = pd.to_datetime(original_df['start_time'])
    original_df['end_time'] = pd.to_datetime(original_df['end_time'])
    comparison_df['start_time'] = pd.to_datetime(comparison_df['start_time'])
    comparison_df['end_time'] = pd.to_datetime(comparison_df['end_time'])

    # Prepare output DataFrame
    output = []

    # Process each group separately
    grouped_orig = original_df.groupby('ID')
    grouped_comp = comparison_df.groupby('ID')

    # Iterate through each group in the original dataframe
    for id, group_orig in grouped_orig:
        # If the ID is not in comparison group, continue with empty comparison group
        group_comp = grouped_comp.get_group(id) if id in grouped_comp.groups else pd.DataFrame()

        for index, row in group_orig.iterrows():
            if not group_comp.empty:
                overlapping_episodes = overlaps(row, group_comp)
                detected_lv1 = overlapping_episodes[overlapping_episodes['lv1']].shape[0] > 0 if row['lv1'] else False
                detected_lv2 = overlapping_episodes[overlapping_episodes['lv2']].shape[0] > 0 if row['lv2'] else False
            else:
                detected_lv1 = False
                detected_lv2 = False

            correct_lv1 = detected_lv1 == row['lv1']
            correct_lv2 = detected_lv2 == row['lv2']

            output.append({
                'ID': row['ID'],
                'start_time': row['start_time'],
                'end_time': row['end_time'],
                'lv1': row['lv1'],
                'lv2': row['lv2'],
                'detected_lv1': detected_lv1,
                'detected_lv2': detected_lv2,
                'correct_lv1': correct_lv1,
                'correct_lv2': correct_lv2
            })

    output_df = pd.DataFrame(output)
    return output_df.sort_values(by=['ID', 'start_time'])

# Example usage:
# original_df = pd.DataFrame(your_original_data)
# comparison_df = pd.DataFrame(your_comparison_data)
# results_df = analyze_episodes_grouped(original_df, comparison_df)


In [ ]:
results_df = analyze_episodes(original_df_expanded, cut_df_expanded)


In [ ]:
results_df


,ID,start_time,end_time,lv1,lv2,detected_lv1,detected_lv2,correct_lv1,correct_lv2
0,helm_1,2020-05-12 04:01:19,2020-05-12 04:21:19,True,False,True,False,True,True
1,helm_1,2020-05-12 04:22:19,2020-05-12 04:40:19,False,False,False,False,True,True
2,helm_1,2020-05-12 04:41:19,2020-05-12 06:06:20,True,False,True,False,True,True
3,helm_1,2020-05-12 06:07:20,2020-05-12 20:20:21,False,False,False,False,True,True
4,helm_1,2020-05-12 20:21:21,2020-05-12 20:41:21,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...
21730,helm_988,2021-04-15 10:26:36,2021-04-15 11:06:35,True,False,True,False,True,True
21731,helm_988,2021-04-15 11:07:35,2021-04-15 18:55:36,False,False,False,False,True,True
21732,helm_988,2021-04-15 18:56:36,2021-04-15 19:26:36,True,False,True,False,True,True
21733,helm_988,2021-04-15 19:27:36,2021-04-16 17:40:39,False,False,False,False,True,True


In [ ]:
original_df_expanded

,ID,start_time,end_time,duration,lv1,lv2
0,helm_1,2020-05-12 04:01:19,2020-05-12 04:21:19,0 days 00:20:00,True,False
1,helm_1,2020-05-12 04:22:19,2020-05-12 04:40:19,0 days 00:18:00,False,False
2,helm_1,2020-05-12 04:41:19,2020-05-12 06:06:20,0 days 01:25:01,True,False
3,helm_1,2020-05-12 06:07:20,2020-05-12 20:20:21,0 days 14:13:01,False,False
4,helm_1,2020-05-12 20:21:21,2020-05-12 20:41:21,0 days 00:20:00,True,False
...,...,...,...,...,...,...
21730,helm_988,2021-04-15 10:26:36,2021-04-15 11:06:35,0 days 00:39:59,True,False
21731,helm_988,2021-04-15 11:07:35,2021-04-15 18:55:36,0 days 07:48:01,False,False
21732,helm_988,2021-04-15 18:56:36,2021-04-15 19:26:36,0 days 00:30:00,True,False
21733,helm_988,2021-04-15 19:27:36,2021-04-16 17:40:39,0 days 22:13:03,False,False


In [ ]:
cut_df_expanded

,ID,start_time,end_time,duration,lv1,lv2
0,helm_1,2020-05-12 04:01:19,2020-05-12 06:16:20,0 days 02:15:01,True,False
1,helm_1,2020-05-12 06:17:20,2020-05-18 06:00:33,5 days 23:43:13,True,False
2,helm_1,2020-05-18 06:01:33,2020-05-18 06:31:34,0 days 00:30:01,True,False
3,helm_1,2020-05-18 06:32:34,2020-05-18 22:45:35,0 days 16:13:01,True,False
4,helm_1,2020-05-18 22:46:35,2020-05-18 23:16:35,0 days 00:30:00,True,False
...,...,...,...,...,...,...
17463,helm_988,2021-04-15 10:31:35,2021-04-15 11:16:35,0 days 00:45:00,True,False
17464,helm_988,2021-04-15 11:17:35,2021-04-15 19:00:36,0 days 07:43:01,True,False
17465,helm_988,2021-04-15 19:01:36,2021-04-15 19:31:36,0 days 00:30:00,True,False
17466,helm_988,2021-04-15 19:32:36,2021-04-16 17:45:39,0 days 22:13:03,True,False


In [67]:
import pandas as pd

class UnionFind:
    def __init__(self):
        self.parent = {}
    
    def find(self, item):
        if self.parent[item] != item:
            self.parent[item] = self.find(self.parent[item])
        return self.parent[item]
    
    def union(self, set1, set2):
        root1 = self.find(set1)
        root2 = self.find(set2)
        if root1 != root2:
            self.parent[root2] = root1
    
    def add(self, item):
        if item not in self.parent:
            self.parent[item] = item

def full_outer_join_custom_per_id(original_df_expanded, cut_df):
    uf = UnionFind()
    overlap_counter = 1
    all_rows = []

    # Initialize Union-Find for all unique identifiers
    for df in [original_df_expanded, cut_df]:
        for idx, row in df.iterrows():
            uf.add((row['ID'], row['unique_id_orig'] if 'unique_id_orig' in row else row['unique_id_cut']))

    # Merge sets where overlaps occur
    for id in set(original_df_expanded['ID']).union(set(cut_df['ID'])):
        orig_group = original_df_expanded[original_df_expanded['ID'] == id]
        cut_group = cut_df[cut_df['ID'] == id]

        for _, orig_row in orig_group.iterrows():
            for _, cut_row in cut_group.iterrows():
                if orig_row['end_time'] >= cut_row['start_time'] and orig_row['start_time'] <= cut_row['end_time']:
                    uf.union((id, orig_row['unique_id_orig']), (id, cut_row['unique_id_cut']))

    # Assign overlap IDs based on connected components
    overlap_id_map = {}
    for key in uf.parent:
        root = uf.find(key)
        if root not in overlap_id_map:
            overlap_id_map[root] = overlap_counter
            overlap_counter += 1

        entry = {'ID': key[0], 'original_id': None, 'cut_id': None,
                 'original_start_time': None, 'original_end_time': None,
                 'original_lv1': None, 'original_lv2': None,
                 'cut_start_time': None, 'cut_end_time': None,
                 'cut_lv1': None, 'cut_lv2': None, 'overlap': False,
                 'overlap_id': overlap_id_map[root]}
        
        if 'original_id' in key:
            entry.update({k: v for k, v in orig_group[orig_group['unique_id_orig'] == key[1]].iloc[0].items() if k in entry})
            entry['original_id'] = key[1]
            entry['overlap'] = True
        if 'cut_id' in key:
            entry.update({k: v for k, v in cut_group[cut_group['unique_id_cut'] == key[1]].iloc[0].items() if k in entry})
            entry['cut_id'] = key[1]
            entry['overlap'] = True

        all_rows.append(entry)

    # Convert the list to a DataFrame
    result_df = pd.DataFrame(all_rows)

    # Convert IDs to integers, handling NaN where necessary
    result_df['original_id'] = result_df['original_id'].astype('Int64')
    result_df['cut_id'] = result_df['cut_id'].astype('Int64')

    return result_df


In [ ]:
results_df = check_overlaps(original_df_expanded, cut_df_expanded)

In [ ]:
results_df

,ID,Original Start,Original End,LV1,LV2,Overlap Type,Detected Start,Detected End
0,helm_1,2020-05-12 04:01:19,2020-05-12 04:21:19,True,False,Full Overlap,2020-05-12 04:01:19,2020-05-12 06:16:20
1,helm_1,2020-05-12 04:22:19,2020-05-12 04:40:19,False,False,Full Overlap,2020-05-12 04:01:19,2020-05-12 06:16:20
2,helm_1,2020-05-12 04:41:19,2020-05-12 06:06:20,True,False,Full Overlap,2020-05-12 04:01:19,2020-05-12 06:16:20
3,helm_1,2020-05-12 06:07:20,2020-05-12 20:20:21,False,False,Partial Overlap,"2020-05-12 04:01:19,2020-05-12 06:17:20","2020-05-12 06:16:20,2020-05-18 06:00:33"
4,helm_1,2020-05-12 20:21:21,2020-05-12 20:41:21,True,False,Full Overlap,2020-05-12 06:17:20,2020-05-18 06:00:33
...,...,...,...,...,...,...,...,...
21730,helm_988,2021-04-15 10:26:36,2021-04-15 11:06:35,True,False,Partial Overlap,"2021-04-15 08:47:36,2021-04-15 10:31:35","2021-04-15 10:30:35,2021-04-15 11:16:35"
21731,helm_988,2021-04-15 11:07:35,2021-04-15 18:55:36,False,False,Partial Overlap,"2021-04-15 10:31:35,2021-04-15 11:17:35","2021-04-15 11:16:35,2021-04-15 19:00:36"
21732,helm_988,2021-04-15 18:56:36,2021-04-15 19:26:36,True,False,Partial Overlap,"2021-04-15 11:17:35,2021-04-15 19:01:36","2021-04-15 19:00:36,2021-04-15 19:31:36"
21733,helm_988,2021-04-15 19:27:36,2021-04-16 17:40:39,False,False,Partial Overlap,"2021-04-15 19:01:36,2021-04-15 19:32:36","2021-04-15 19:31:36,2021-04-16 17:45:39"


### COncat approach

In [109]:
import pandas as pd

def create_overlap_ids(original_df, cut_df):
    # Standardize column names if they differ
    original_df.rename(columns={'original_start_time': 'start_time', 'original_end_time': 'end_time'}, inplace=True)
    cut_df.rename(columns={'cut_start_time': 'start_time', 'cut_end_time': 'end_time'}, inplace=True)
    
    # Adding a marker to distinguish between original and cut
    original_df['source'] = 'original'
    cut_df['source'] = 'cut'
    
    # Concatenate the DataFrames
    full_df = pd.concat([original_df, cut_df], ignore_index=True)
    
    # Sort by ID and then by start_time
    full_df.sort_values(by=['ID', 'start_time'], inplace=True)

    # Prepare a DataFrame to collect results
    results_df = pd.DataFrame()

    # Group by ID and process each group independently
    for group_id, group_df in full_df.groupby('ID'):
        overlap_windows = {}
        overlap_ids = []
        current_overlap_id = 1

        # Iterate through rows within the group sorted by start time
        for index, row in group_df.iterrows():
            current_start = row['start_time']
            current_end = row['end_time']
            found_overlap = False

            # Check for overlap within existing windows
            for overlap_id, (earliest_start, latest_end) in overlap_windows.items():
                if current_start <= latest_end and current_end >= earliest_start:
                    overlap_ids.append(overlap_id)
                    overlap_windows[overlap_id] = (min(earliest_start, current_start), max(latest_end, current_end))
                    found_overlap = True
                    break

            if not found_overlap:
                overlap_ids.append(current_overlap_id)
                overlap_windows[current_overlap_id] = (current_start, current_end)
                current_overlap_id += 1

        # Assign overlap_ids to the group DataFrame
        group_df['overlap_id'] = overlap_ids
        results_df = pd.concat([results_df, group_df], ignore_index=True)

    # Select only the specified columns
    results_df = results_df[['overlap_id', 'ID', 'source', 'start_time', 'end_time', 'lv1', 'lv2']]
    
    # Calculate duration and add it to the DataFrame
    results_df['duration'] = results_df['end_time'] - results_df['start_time']
    
    return results_df

# Example usage:
# Assuming original_df_expanded and cut_df are already loaded and preprocessed
# merged_df = create_overlap_ids(original_df_expanded, cut_df)
# print(merged_df


In [110]:
result_df = create_overlap_ids(original_df_expanded, cut_df)


In [105]:
cut_df['lv1'] = True

In [112]:
result_df['group_id'] = result_df['ID'].astype(str) + '_' + result_df['overlap_id'].astype(str)

In [113]:
result_df

,overlap_id,ID,source,start_time,end_time,lv1,lv2,duration,group_id
0,1,helm_1,original,2020-05-12 04:01:19,2020-05-12 04:21:19,True,False,0 days 00:20:00,helm_1_1
1,1,helm_1,cut,2020-05-12 04:01:19,2020-05-12 06:16:20,True,False,0 days 02:15:01,helm_1_1
2,1,helm_1,original,2020-05-12 04:22:19,2020-05-12 04:40:19,False,False,0 days 00:18:00,helm_1_1
3,1,helm_1,original,2020-05-12 04:41:19,2020-05-12 06:06:20,True,False,0 days 01:25:01,helm_1_1
4,1,helm_1,original,2020-05-12 06:07:20,2020-05-12 20:20:21,False,False,0 days 14:13:01,helm_1_1
...,...,...,...,...,...,...,...,...,...
30705,47,helm_988,original,2021-04-15 18:56:36,2021-04-15 19:26:36,True,False,0 days 00:30:00,helm_988_47
30706,47,helm_988,cut,2021-04-15 19:01:36,2021-04-15 19:31:36,True,False,0 days 00:30:00,helm_988_47
30707,47,helm_988,original,2021-04-15 19:27:36,2021-04-16 17:40:39,False,False,0 days 22:13:03,helm_988_47
30708,48,helm_988,original,2021-04-16 17:41:39,2021-04-16 18:41:39,True,False,0 days 01:00:00,helm_988_48


In [102]:
result_df.sort_values('overlap_id')

,ID,start_time,end_time,duration,lv1,lv2,unique_id_orig,type,source,Unnamed: 1,prolonged,unique_id_cut,overlap_id
0,helm_1,2020-05-12 04:01:19,2020-05-12 04:21:19,0 days 00:20:00,True,False,1.0,original,original,NaN,NaN,NaN,1
14843,helm_425,2020-05-21 18:01:42,2020-05-21 19:26:42,0 days 01:25:00,True,True,14844.0,original,original,NaN,NaN,NaN,1
14842,helm_425,2020-05-21 13:37:42,2020-05-21 18:00:42,0 days 04:23:00,False,False,14843.0,original,original,NaN,NaN,NaN,1
27884,helm_425,2020-05-21 12:16:42,2020-05-21 13:46:42,0 days 01:30:00,NaN,True,NaN,cut,cut,22.0,False,6150.0,1
14841,helm_425,2020-05-21 12:16:42,2020-05-21 13:36:42,0 days 01:20:00,True,True,14842.0,original,original,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18614,helm_707,2019-05-23 02:33:32,2019-05-26 21:21:25,3 days 18:47:53,False,False,18615.0,original,original,NaN,NaN,NaN,1157
18615,helm_707,2019-05-26 21:22:25,2019-05-26 21:52:25,0 days 00:30:00,True,True,18616.0,original,original,NaN,NaN,NaN,1158
18616,helm_707,2019-05-26 21:53:25,2019-05-27 12:36:24,0 days 14:42:59,False,False,18617.0,original,original,NaN,NaN,NaN,1159
18617,helm_707,2019-05-27 12:37:24,2019-05-27 13:17:24,0 days 00:40:00,True,False,18618.0,original,original,NaN,NaN,NaN,1160
